# pandas vs polars for pyglider performance

This script compares the wall time, memory usage and disk space utilisation of the original pandas/netcdf and the proposed polars/parquet methods for loading and merging SeaExplorer nav and pld files in pyglider.

We use the inbuilt python c profiler and [memory profiler](https://github.com/pythonprofilers/memory_profiler) from pip

We compare the current pyglider, henceforth *pandas*
```
commit 524d6e59a1a43e8706c7ed68558dac6288cb3094 (HEAD -> main, origin/main, origin/HEAD)
Merge: 48ae4d5 95ec8ca
Author: hvdosser <46034533+hvdosser@users.noreply.github.com>
Date:   Tue Sep 27 14:31:28 2022 -0700
```
With the version in PR #120, henceforth *polars*:
```
commit eaf2450ede8ec8beb8805933525e524d20082309 (HEAD -> callum-patch-33, origin/callum-patch-33)
Author: Callum Rollo <c.rollo@outlook.com>
Date:   Fri Oct 7 10:03:06 2022 +0200
```

### nrt comparison

input data:

In [20]:
!ls sea63_35_nrt | wc -l
! du -h sea63_35_nrt

322
6,4M	sea63_35_nrt


In [2]:
!/home/callum/anaconda3/envs/pyglider/bin/python -m cProfile -s cumulative test_scripts/nrt_polars.py | head -n 20

Maximum memory usage: 166.921875
         3197759 function calls (3146702 primitive calls) in 4.869 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1791/1    0.023    0.000    4.870    4.870 {built-in method builtins.exec}
        1    0.000    0.000    4.870    4.870 nrt_polars.py:1(<module>)
        1    0.000    0.000    3.118    3.118 memory_profiler.py:267(memory_usage)
        1    0.012    0.012    3.111    3.111 seaexplorer.py:41(raw_to_rawnc)
      324    0.001    0.000    2.871    0.009 frame.py:2070(write_parquet)
      324    2.869    0.009    2.869    0.009 {method 'write_parquet' of 'builtins.PyDataFrame' objects}
      176    0.004    0.000    2.472    0.014 __init__.py:1(<module>)
   1761/4    0.005    0.000    0.886    0.222 <frozen importlib._bootstrap>:1022(_find_and_load)
   1729/4    0.004    0.000    0.886    0.222 <frozen importlib._bootstrap>:987(_find_and_load_unlocked)
   1634/5    0.004    0.

In [6]:
!du -h sea63_35_nrt_polars

7,6M	sea63_35_nrt_polars


In [5]:
!/home/callum/anaconda3/envs/pyglider/bin/python -m cProfile -s cumulative test_scripts/nrt_pandas.py | head -n 20

Maximum memory usage: 159.5546875
         50057574 function calls (49431678 primitive calls) in 24.836 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1741/1    0.023    0.000   24.850   24.850 {built-in method builtins.exec}
        1    0.000    0.000   24.850   24.850 nrt_pandas.py:1(<module>)
        1    0.000    0.000   15.888   15.888 memory_profiler.py:267(memory_usage)
        1    0.026    0.026   15.882   15.882 seaexplorer.py:41(raw_to_rawnc)
        1    0.012    0.012    7.280    7.280 seaexplorer.py:190(merge_rawnc)
     8746    0.072    0.000    6.003    0.001 merge.py:696(merge_core)
      325    0.004    0.000    5.854    0.018 dataset.py:1797(to_netcdf)
      325    0.007    0.000    5.846    0.018 api.py:1130(to_netcdf)
      325    0.003    0.000    5.515    0.017 api.py:1251(dump_to_store)
     8101    0.022    0.000    5.414    0.001 dataset.py:1423(__setitem__)
      325    0.001    0.000    5.

In [7]:
!du -h sea63_35_nrt_pandas

28M	sea63_35_nrt_pandas


| resource | pandas | polars |
| --- | --- | --- |
| CPU time | 24.8 seconds | 4.8 seconds |
| Memory | 159.5 MB | 166.9 MB |
| Disk usage | 28 MB | 7.6 MB |

### Complete dataset

In [19]:
!ls sea63_35_nrt_full | wc -l
!du -h sea63_35_nrt_full

2424
83M	sea63_35_nrt_full


In [9]:
!/home/callum/anaconda3/envs/pyglider/bin/python -m cProfile -s cumulative test_scripts/raw_pandas.py | head -n 20

Maximum memory usage: 200.1328125
         990324578 function calls (985763199 primitive calls) in 511.913 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1741/1    0.023    0.000  512.019  512.019 {built-in method builtins.exec}
        1    0.009    0.009  512.019  512.019 raw_pandas.py:1(<module>)
        1    0.000    0.000  379.167  379.167 memory_profiler.py:267(memory_usage)
        1    0.239    0.239  379.158  379.158 seaexplorer.py:41(raw_to_rawnc)
     7273    0.075    0.000  273.834    0.038 datetimes.py:706(to_datetime)
     7273    0.044    0.000  255.665    0.035 datetimes.py:326(_convert_listlike_datetimes)
     3636    0.030    0.000  254.912    0.070 datetimes.py:2130(objects_to_datetime64ns)
     3636   10.425    0.003  254.826    0.070 {built-in method pandas._libs.tslib.array_to_datetime}
9775/7351    0.028    0.000  248.231    0.034 _decorators.py:308(wrapper)
     2424    0.008    0.000  248.117 

In [10]:
!du -h sea63_35_raw_pandas

990M	sea63_35_raw_pandas


In [14]:
!/home/callum/anaconda3/envs/pyglider/bin/python -m cProfile -s cumulative test_scripts/raw_polars.py | head -n 20

Maximum memory usage: 191.2109375
         10682910 function calls (10514816 primitive calls) in 45.921 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1791/1    0.023    0.000   45.922   45.922 {built-in method builtins.exec}
        1    0.025    0.025   45.922   45.922 raw_polars.py:1(<module>)
        1    0.000    0.000   30.204   30.204 memory_profiler.py:267(memory_usage)
        1    0.123    0.123   30.195   30.195 seaexplorer.py:41(raw_to_rawnc)
     2426    0.009    0.000   27.148    0.011 frame.py:2070(write_parquet)
     2426   27.129    0.011   27.129    0.011 {method 'write_parquet' of 'builtins.PyDataFrame' objects}
        1    0.109    0.109   11.378   11.378 seaexplorer.py:278(raw_to_timeseries)
        1    0.403    0.403    6.052    6.052 utils.py:132(get_profiles_new)
2430/2428    0.009    0.000    5.182    0.002 utils.py:292(wrapper)
11894/11888    0.026    0.000    4.995    0.000 dataarray.py:80

In [15]:
!du -h sea63_35_raw_polars

161M	sea63_35_raw_polars


# Performance comparison

| resource | pandas | polars |
| --- | --- | --- |
| CPU time | 511.9 seconds | 45.9 seconds |
| Memory | 200.1 MB | 191.2 MB |
| Disk usage | 990 MB | 161 MB |

Using polars in place of pandas results in a decrease in processing times of 5-10 X, similar memory usage of +/-10 % and decreased storage space of intermediate products of 5 X.

Profiling could be done on the codebase more broadly to identify other bottlenecks

# Compare resulting products

In [3]:
import pandas as pd
import polars as pl
import xarray as xr

In [4]:
!ls

mprofile_20221007140216.dat  sea63_35_nrt_polars
sea63_35_nrt		     sea63_35_nrt_rawnc
sea63_35_nrt_full	     sea63_35_nrt_rawnc_complete
sea63_35_nrt_intermediate    sea63_35_raw_pandas
sea63_35_nrt_l0		     sea63_35_raw_polars
sea63_35_nrt_l00	     sea63_35_raw_polars_l0
sea63_35_nrt_l0_pandas	     test_scripts
sea63_35_nrt_l0_polars	     Untitled.ipynb
sea63_35_nrt_pandas


In [5]:
ds_pandas = xr.open_dataset("sea63_35_nrt_pandas/sea063.0035.gli.sub.0007.nc")

In [7]:
df_polars = pl.read_parquet("sea63_35_nrt_polars/sea063.0035.gli.sub.0007.parquet")

In [10]:
ds_pandas.to_dataframe()

,NavState,SecurityLevel,Heading,Declination,Pitch,Roll,Depth,Temperature,Pa,Lat,...,BallastCmd,BallastPos,LinCmd,LinPos,AngCmd,AngPos,Voltage,Altitude,Unnamed: 22,fnum
time,,,,,,,,,,,,,,,,,,,,,
2022-02-27 12:14:47,117,32,216.17,6,59.86,8.01,4.2,6.0,73560,5533.312,...,500,506.5,5.0,3.6,0,1.4,28.9,-1.0,NaN,7.0
2022-02-27 12:14:51,115,32,216.41,6,60.43,8.71,1.2,6.0,73555,5533.311,...,500,506.5,100.0,16.2,0,0.9,28.9,-1.0,NaN,7.0
2022-02-27 12:15:01,115,32,217.04,6,-0.88,3.34,-0.1,6.0,73539,5533.311,...,500,506.5,100.0,88.4,0,0.9,28.9,-1.0,NaN,7.0
2022-02-27 12:15:11,115,32,215.64,6,-39.47,3.55,0.2,6.0,73524,5533.310,...,500,504.8,100.0,101.3,0,1.4,28.9,-1.0,NaN,7.0
2022-02-27 12:15:31,116,32,212.25,6,-35.52,7.16,-0.0,6.0,73522,5533.307,...,500,506.5,100.0,101.6,0,1.4,28.9,-1.0,NaN,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-27 12:41:56,117,32,228.40,6,-3.65,5.68,54.5,6.3,73849,5533.204,...,500,504.8,5.0,23.4,0,0.9,28.9,-1.0,NaN,7.0
2022-02-27 12:42:16,117,32,224.27,6,58.62,7.70,51.2,6.3,73783,5533.203,...,500,506.5,5.0,3.7,0,0.9,28.9,-1.0,NaN,7.0
2022-02-27 12:42:36,117,32,228.59,6,59.51,9.45,40.7,6.3,73711,5533.199,...,500,506.5,5.0,3.7,0,1.4,28.9,-1.0,NaN,7.0


In [9]:
df_polars

time,NavState,SecurityLevel,Heading,Declination,Pitch,Roll,Depth,Temperature,Pa,Lat,Lon,DeadReckoning,DesiredH,BallastCmd,BallastPos,LinCmd,LinPos,AngCmd,AngPos,Voltage,Altitude,fnum
datetime[μs],i64,i64,f64,i64,f64,f64,f64,f64,i64,f64,f64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,i64
2022-02-27 12:14:47,117,32,216.17,6,59.86,8.01,4.2,6.0,73560,5533.312,1622.367,1,228,500,506.5,5.0,3.6,0,1.4,28.9,-1.0,7
2022-02-27 12:14:51,115,32,216.41,6,60.43,8.71,1.2,6.0,73555,5533.311,1622.365,1,228,500,506.5,100.0,16.2,0,0.9,28.9,-1.0,7
2022-02-27 12:15:01,115,32,217.04,6,-0.88,3.34,-0.1,6.0,73539,5533.311,1622.364,1,228,500,506.5,100.0,88.4,0,0.9,28.9,-1.0,7
2022-02-27 12:15:11,115,32,215.64,6,-39.47,3.55,0.2,6.0,73524,5533.31,1622.364,1,228,500,504.8,100.0,101.3,0,1.4,28.9,-1.0,7
2022-02-27 12:15:31,116,32,212.25,6,-35.52,7.16,-0.0,6.0,73522,5533.307,1622.361,0,228,500,506.5,100.0,101.6,0,1.4,28.9,-1.0,7
2022-02-27 12:16:00,116,32,210.62,6,-56.01,7.49,-0.0,6.0,73520,5533.306,1622.363,0,228,500,506.5,100.0,101.5,0,1.4,28.9,-1.0,7
2022-02-27 12:16:21,116,32,208.71,6,-62.97,10.43,0.3,6.0,73519,5533.305,1622.363,0,228,500,508.1,100.0,101.6,0,1.4,28.9,-1.0,7
2022-02-27 12:16:41,116,32,209.59,6,-59.65,10.11,0.0,5.9,73510,5533.304,1622.366,0,231,500,508.1,100.0,101.6,0,1.4,28.9,-1.0,7
2022-02-27 12:17:01,116,32,204.84,6,-58.46,7.16,0.0,5.9,73507,5533.303,1622.368,0,231,500,508.1,100.0,101.5,0,1.4,28.9,-1.0,7


In [12]:
ds_pandas = xr.open_dataset("sea63_35_nrt_l0_pandas/SAMBA.nc")

In [14]:
ds_polars = xr.open_dataset("sea63_35_nrt_l0_polars/SAMBA.nc")

In [16]:
ds_pandas

<xarray.Dataset>
Dimensions:                   (time: 8271)
Coordinates:
    latitude                  (time) float64 55.57 55.57 55.57 ... 55.57 55.57
    longitude                 (time) float64 16.37 16.37 16.37 ... 16.35 16.35
  * time                      (time) datetime64[ns] 2022-02-27T08:35:41.70800...
    depth                     (time) float64 ...
Data variables: (12/42)
    nav_state                 (time) float64 116.0 116.0 116.0 ... 117.0 117.0
    heading                   (time) float64 339.2 340.8 339.3 ... 23.01 21.42
    pitch                     (time) float64 0.1175 1.118 0.7645 ... 18.85 19.33
    roll                      (time) float64 -2.011 0.8471 ... -0.4231 -0.34
    dive_num                  (time) float64 1.0 1.0 1.0 ... 1.211e+03 1.211e+03
    security_level            (time) float64 ...
    ...                        ...
    distance_over_ground      (time) float64 ...
    profile_index             (time) float64 ...
    profile_direction         (time) float64 ...
    potential_density         (time) float64 ...
    density                   (time) float64 ...
    potential_temperature     (time) float64 ...
Attributes: (12/60)
    AD2CP:                     {'make': 'Nortek', 'model': 'AD2CP', 'serial':...
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0
    acknowledgement:           This study used data collected and made freely...
    altimeter:                 {'make': 'UNKNOWN', 'model': 'UNKNOWN', 'seria...
    cdm_data_type:             Trajectory
    ...                        ...
    summary:                    Part of SAMBA continuous monitoring
    time_coverage_end:         2022-03-21T11:02:53.164000000
    time_coverage_start:       2022-02-27T08:35:41.708000000
    title:                     Ljung63-20220227T0835
    transmission_system:       IRIDIUM
    wmo_id:                    6801707

In [17]:
ds_polars

<xarray.Dataset>
Dimensions:                   (time: 8271)
Coordinates:
    latitude                  (time) float64 55.57 55.57 55.57 ... 55.57 55.57
    longitude                 (time) float64 16.37 16.37 16.37 ... 16.35 16.35
  * time                      (time) datetime64[ns] 2022-02-27T08:35:41.70800...
    depth                     (time) float64 ...
Data variables: (12/42)
    nav_state                 (time) float64 116.0 116.0 116.0 ... 117.0 117.0
    heading                   (time) float64 339.2 340.8 339.3 ... 23.01 21.42
    pitch                     (time) float64 0.1175 1.118 0.7645 ... 18.85 19.33
    roll                      (time) float64 -2.011 0.8471 ... -0.4231 -0.34
    dive_num                  (time) float64 1.0 1.0 1.0 ... 1.211e+03 1.211e+03
    security_level            (time) float64 ...
    ...                        ...
    distance_over_ground      (time) float64 ...
    profile_index             (time) float64 ...
    profile_direction         (time) float64 ...
    potential_density         (time) float64 ...
    density                   (time) float64 ...
    potential_temperature     (time) float64 ...
Attributes: (12/60)
    AD2CP:                     {'make': 'Nortek', 'model': 'AD2CP', 'serial':...
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0
    acknowledgement:           This study used data collected and made freely...
    altimeter:                 {'make': 'UNKNOWN', 'model': 'UNKNOWN', 'seria...
    cdm_data_type:             Trajectory
    ...                        ...
    summary:                    Part of SAMBA continuous monitoring
    time_coverage_end:         2022-03-21T11:02:53.164000000
    time_coverage_start:       2022-02-27T08:35:41.708000000
    title:                     Ljung63-20220227T0835
    transmission_system:       IRIDIUM
    wmo_id:                    6801707